## Semantic Router

Semantic router is a clever way to direct the flow of conversation using natural language. <br>
Sometimes you may want to direct a query or user input like how you would using `if` or `elif` in python. <br>
Adding this logic into your prompt can introduce complexity to your system messages that can reduce performance.

You can also use semantic router to act as a guardrail against certain topics.

In [ ]:
import os
import json

from semantic_router import Route
from semantic_router.layer import RouteLayer
from semantic_router.encoders import CohereEncoder, OpenAIEncoder
from semantic_router.utils.function_call import get_schema

from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
encoder = OpenAIEncoder()
# encoder = CohereEncoder()

In [ ]:
# we could use this as a guide for our chatbot to avoid political conversations
politics = Route(
    name="religion",
    utterances=[
        "Do you believe in god?"
        "Is god real?"
        "Which religion are you?"
    ],
)

# this could be used as an indicator to our chatbot to switch to a more
# conversational prompt
chitchat = Route(
    name="animals",
    utterances=[
        "What is the fastest animal in the world?"
        "How many breeds of dogs are there?"
        "Do you have a pet at home?"
        "What the difference between a Bonobo and a Chimpanzee?"
    ],
)

# we place both of our decisions together into single list
routes = [politics, chitchat]

In [ ]:
# Create a router layer
rl = RouteLayer(encoder=encoder, routes=routes)

# Classify the text - `None` if no match
rl("What religion are you?").name

### Saving and loading routers

You can save a load route configurations in a `.json` format.

In [ ]:
# save router
rl.to_json("layer.json")

# load router
loaded_rl = RouteLayer.from_json("layer.json")

### Function Calling
You can also use semantic router for function calling. Which can be beneficial when you don't want to use an LLM for to deciding to ue a function, which can have longer processing times.

In [ ]:
# Example function 
def get_company_info(query: str) -> str:
    return "ABCXYZ Company is a tech company established in 2001 in San Diego."
    
schema = get_schema(get_company_info)

# dynamic route - Dynamic routes use an LLM
route_with_function = Route(
    name="get_domain_info",
    utterances=[
        "What is ABCXYZ?",
        "When was ABCXYZ created?",
        "Where was ABCXYZ made?",
    ],
    function_schema=schema,
)

# Add the route to the route layer and query
rl.add(route_with_function)
rl("which city did ABCXYZ start up in?")

### Controlling flow

You can control the flow of how the system reacts to a certain semantic input based on the routes you define. For example, this can achieved by either checking the type of query that is being asked and starting up a different process OR it can be used to update system messages for an LLM further downstream in the process.

In [ ]:
# Different processes
route = rl("Did she believe in god?")
if route.name == "religion":
    # DO SOMETHING 
    pass
elif route.name == "animals":
    # DO SOMETHING ELSE
    pass
elif route.name == "get_domain_info":
    # DO SOMETHING ELSE
    pass

In [ ]:
# Dynamic system messages
def semantic_layer(query: str):
    route = rl(query)
    if route.name == "religion":
        query += f"<ADDITIONAL_CONTEXT>"
    elif route.name == "animals":
        query += f"<ADDITIONAL_CONTEXT>"
    elif route.name == "get_domain_info":
        query += f"<ADDITIONAL_CONTEXT>"
    return query

query = "<some_query>"
query = semantic_layer(query)


---

In [ ]:
# Some things are not clearly defined as the project is new
# TODO:
# HybridRouteLayer
# RouteConfig